# Evaluate the best lossy compression for a sample ECG signal using mean absolute error (MAE) and mean squared error (MSE).

In [ ]:
import numpy as np
from scipy.fft import fft, ifft
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Functions for all methods:

In [ ]:
def piecewise_constant_approx(signal, segments):
    n = len(signal)
    seg_len = n // segments
    approx = np.zeros(n)
    for i in range(segments):
        start = i * seg_len
        end = (i + 1) * seg_len if i < segments - 1 else n
        approx[start:end] = np.mean(signal[start:end])
    return approx


def piecewise_linear_approx(signal, segments):
    n = len(signal)
    seg_len = n // segments
    approx = np.zeros(n)
    for i in range(segments):
        start = i * seg_len
        end = (i + 1) * seg_len if i < segments - 1 else n
        x = np.arange(start, end)
        y = signal[start:end]
        if len(x) > 1:
            p = np.polyfit(x, y, 1)
            approx[start:end] = np.polyval(p, x)
        else:
            approx[start:end] = y
    return approx


def quantize_signal(signal, num_levels=32):
    min_val, max_val = np.min(signal), np.max(signal)
    bins = np.linspace(min_val, max_val, num_levels)
    quantized = np.digitize(signal, bins) - 1
    return bins[quantized]


def line_simplify(signal, tolerance=5.0):
    from rdp import rdp
    points = np.column_stack((np.arange(len(signal)), signal))
    simplified = rdp(points, epsilon=tolerance)
    result = np.interp(np.arange(len(signal)), simplified[:, 0], simplified[:, 1])
    return result

# Evaluate with a threshold

In [ ]:
def fft_threshold(signal, keep_ratio=0.1):
    fft_vals = fft(signal)
    mag = np.abs(fft_vals)
    threshold = np.quantile(mag, 1 - keep_ratio)
    fft_vals[mag < threshold] = 0
    return np.real(ifft(fft_vals))


def evaluate_compression(original, compressed):
    mae = mean_absolute_error(original, compressed)
    mse = mean_squared_error(original, compressed)
    return mae, mse

    from compression.lossy_methods import *

# Print results

In [ ]:
# Sample ECG signal
signal = np.array(X_train[0])

# Run all techniques
methods = {
    "PCA": piecewise_constant_approx(signal, segments=50),
    "PLA": piecewise_linear_approx(signal, segments=50),
    "Quant": quantize_signal(signal, num_levels=32),
    "LineSimplify": line_simplify(signal, tolerance=5),
    "FFT": fft_threshold(signal, keep_ratio=0.1),
}

# Evaluate
for name, compressed in methods.items():
    mae, mse = evaluate_compression(signal, compressed)
    print(f"{name}: MAE = {mae:.2f}, MSE = {mse:.2f}")